## Example of fitting some lens systems

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy
import astropy.units as u
from cdfvi import funcs, TDCosmoChi2, TDPolyChi2

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda_v2 = ccl.Cosmology(
    Omega_c=0.94, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_w0wa = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0=-0.8, wa=-1.0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)


In [ ]:
sim_vals = dict(
    n_obj = 1200,
    z_lens_mean = 0.74,
    z_lens_scale = 0.49,
    z_source_mean = 2.31,
    z_source_scale = 0.45,
)

In [ ]:
sims = funcs.simulate_lens_systems(**sim_vals)
z_source_vals = sims.z_source.to_numpy()
z_lens_vals = sims.z_lens.to_numpy()

In [ ]:
R_vals = funcs.r_flat(cosmo, z_lens_vals, z_source_vals)
R_vals_no_lambda = funcs.r_flat(cosmo_no_lambda, z_lens_vals, z_source_vals)
R_vals_wowa = funcs.r_flat(cosmo_w0wa, z_lens_vals, z_source_vals)

err_level = 0.05
scatter = np.random.normal(0, err_level, size=len(z_source_vals))
R_errs = R_vals*err_level
R_errs_no_lambda = R_vals_no_lambda*err_level
R_errs_wowa = R_vals_wowa*err_level

R_obs = R_vals*(1+scatter)

In [ ]:
zmin = 0.01
zmax = 6
nzBins = 300
z_grid = np.linspace(zmin, zmax, nzBins)
z_widths = z_grid[1:] - z_grid[0:-1]
a_grid = funcs.z_to_a(z_grid)

In [ ]:
result1d = np.polynomial.Polynomial.fit(z_grid[1:], funcs.j_integral_simple(cosmo, z_grid), deg=6).convert()
int_j_func = np.polynomial.Polynomial(result1d.coef)

In [ ]:
delta = funcs.function_ratio(int_j_func, z_source_vals, z_lens_vals) - funcs.chi_ratio(cosmo, z_lens_vals, z_source_vals)

In [ ]:
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo, z_grid))
_ = plt.plot(z_grid, result1d(z_grid))
_ = plt.plot(z_grid, int_j_func(z_grid))

In [ ]:
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo, z_grid) - result1d(z_grid[1:]))
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo, z_grid) - int_j_func(z_grid[1:]))

In [ ]:
#fixed_vals = dict(H0=70. * u.km / u.s / u.Mpc)
#param_names = ['Om0', 'Ob0']
#param_scales = dict()
#init_pars_cosmo = [0.30, 0.05]

fixed_vals = dict()
param_names = ['H0', 'Om0', 'Ob0']
init_pars_cosmo = [70, 0.25, 0.05]
param_scales = dict(H0=u.km / u.s / u.Mpc)

In [ ]:
pf = TDPolyChi2(z_lens_vals, z_source_vals, R_obs, R_errs, constraint=1e-6)
pf_unc = TDPolyChi2(z_lens_vals, z_source_vals, R_obs, R_errs)
cf = TDCosmoChi2(z_lens_vals, z_source_vals, R_obs, R_errs, param_names, param_scales, **fixed_vals)

In [ ]:
init_pars = result1d.coef

In [ ]:
result = scipy.optimize.minimize(pf, init_pars, method='nelder-mead', options={'xatol': 1e-9, 'disp': True, 'maxiter':2000})
result_unc = scipy.optimize.minimize(pf_unc, init_pars, method='nelder-mead', options={'xatol': 1e-9, 'disp': True, 'maxiter':2000})
result_c = scipy.optimize.minimize(cf, init_pars_cosmo, method='L-BFGS-B', options={'gtol': 0.01})

In [ ]:
mat = np.matrix(result_c.hess_inv.todense())
hesse = mat.getI()
h0_grid = np.linspace(50, 100, 501)
Om0_grid = np.linspace(0.2, 0.5, 501)
mesh = np.meshgrid(h0_grid, Om0_grid)
dx, dy = mesh[0] - result_c.x[0], mesh[1] - result_c.x[1]
nulls = np.zeros(dx.shape)
dels = np.squeeze(np.dstack([dx.ravel(), dy.ravel(), nulls.ravel()]))
diffs = np.reshape(np.array([np.matmul(del_, np.matmul(hesse, del_).T) for del_ in dels]), dx.shape)

In [ ]:
_ = plt.contour(h0_grid, Om0_grid, diffs, levels=[1, 4, 9])
_ = plt.xlabel(r'$H_{0} [km \cdot s^{-1} \cdot Mpc^{-1}]$') 
_ = plt.ylabel(r'$\Omega_{m, 0}$')

In [ ]:
_ = plt.scatter(pf.z_d, pf.z_s, c=pf.resid(init_pars))
_ = plt.colorbar()

In [ ]:
_ = plt.scatter(cf.z_d, cf.z_s, c=cf.resid(result_c.x), s=5)
_ = plt.colorbar(label=r"$\delta R$")
_ = plt.xlabel(r"$z_d$")
_ = plt.ylabel(r"$z_s$")
_ = plt.ylim(0, 6)
_ = plt.xlim(0, 3)

In [ ]:
bin_edges = np.logspace(-0.10, 0.5, 10)
z_source_vals = cf.z_s
z_lens_vals = cf.z_d
resid_vals = cf.resid(result_c.x)
err_vals = cf.errs
for bin_low, bin_hi in zip(bin_edges[0:-1], bin_edges[1:]):
    source_mask = np.bitwise_and(z_source_vals > bin_low, z_source_vals <= bin_hi)
    _ = plt.errorbar(
        z_lens_vals[source_mask],
        resid_vals[source_mask], 
        err_vals[source_mask],
        marker='.',
        ls="", 
        label=f"z_s {bin_low:.2f} - {bin_hi:.2f}",
        alpha=0.4,
    )
_ = plt.xlabel(r"$z_{d}$")
_ = plt.ylabel(r"$\delta R_{ds}$")
_ = plt.xlim(0, 2.5)
_ = plt.ylim(-0.2, 0.2)


In [ ]:
modeler = np.polynomial.Polynomial(result_c.x)
modeler_unc = np.polynomial.Polynomial(result_c.x)

In [ ]:
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo, z_grid))
_ = plt.plot(z_grid, modeler(z_grid))
_ = plt.plot(z_grid, modeler_unc(z_grid))

In [ ]:
modelSpace = pf.model_space(z_grid, result_c.x)

In [ ]:
_ = plt.imshow(modelSpace, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{ds} = \chi_{s}/\chi_{d}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')

In [ ]:
j_int = np.polynomial.Polynomial(init_pars)
j_int2 = np.polynomial.Polynomial(result.x)
j_int_unc = np.polynomial.Polynomial(result_unc.x)

In [ ]:
j_z = j_int.deriv(1)
j_z2 = j_int2.deriv(1)
j_int_unc = j_int_unc.deriv(1)

In [ ]:
_ = plt.plot(z_grid, 1/j_z(z_grid))
_ = plt.plot(z_grid, 1/j_z2(z_grid))
_ = plt.plot(z_grid, cosmo.h_over_h0(a_grid))
#_ = plt.xlim(0, 3)

In [ ]:
_ = plt.plot(z_grid, (1/j_z(z_grid))/cosmo.h_over_h0(a_grid))